In [6]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# Read the cities data

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Fayaoue,NC,2022-03-01 02:11:40,-20.6500,166.5333,82.80,84,96,27.22
1,1,Ilulissat,GL,2022-03-01 02:11:40,69.2167,-51.1000,7.79,57,65,11.30
2,2,Havre-Saint-Pierre,CA,2022-03-01 02:09:41,50.2334,-63.5986,-6.14,90,6,8.97
3,3,Georgetown,MY,2022-03-01 02:08:05,5.4112,100.3354,78.75,80,95,4.68
4,4,Garowe,SO,2022-03-01 02:11:41,8.4054,48.4845,65.70,87,94,10.60


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Temperature Heat Map

In [9]:
# Get the maximum temperature. Prevents the max tempearture for being negative by only allowing positve values
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [19]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Assign the figure variable.
fig = gmaps.figure(center=(30,0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations,
    weights=[max(temp, 0) for temp in max_temp],
    dissipating=False,
    max_intensity=300,
    point_radius=3)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

# Humdity Heatmap

In [20]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

# Cloudiness Heatmap

In [21]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
cloud = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloud, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

# Windspeed Heatmap

In [22]:
# Heatmap of Windspeeds
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

# Prefered Cities by temperature

In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [40]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()


# Hotels in preferred cities

In [43]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Fayaoue,NC,82.80,-20.6500,166.5333,
3,Georgetown,MY,78.75,5.4112,100.3354,
4,Garowe,SO,65.70,8.4054,48.4845,
10,Vao,NC,79.95,-22.6667,167.4833,
11,Hilo,US,76.80,19.7297,-155.0900,
12,Busselton,AU,74.43,-33.6500,115.3333,
13,Cayenne,GF,76.14,4.9333,-52.3333,
14,Rikitea,PF,79.32,-23.1203,-134.9692,
15,Kapaa,US,80.58,22.0752,-159.3190,
25,Avarua,CK,84.25,-21.2078,-159.7750,


In [44]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [46]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not found")
        continue

Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found
Hotel Not found


In [50]:
# Write data to csv file for later loading, prevents having to hit API again
# hotel_df.to_csv("./weather_data/hotels.csv",index_label="Hotel_ID")

# Heat map of preferred hotels

In [2]:
# Read the hotel csv file above
# hotel_df = pd.read_csv("./weather_data/hotels.csv")

In [13]:
hotel_df.head(10)

,Hotel_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,0,Fayaoue,NC,82.80,-20.6500,166.5333,Chez Dydyce Guesthouse
1,3,Georgetown,MY,78.75,5.4112,100.3354,Cititel Penang
2,4,Garowe,SO,65.70,8.4054,48.4845,Curubo Hotel
3,10,Vao,NC,79.95,-22.6667,167.4833,Hotel Kou-Bugny
4,11,Hilo,US,76.80,19.7297,-155.0900,Hilo Hawaiian Hotel
5,12,Busselton,AU,74.43,-33.6500,115.3333,Observatory Guest House
6,13,Cayenne,GF,76.14,4.9333,-52.3333,Hôtel Le Dronmi
7,14,Rikitea,PF,79.32,-23.1203,-134.9692,People ThankYou
8,15,Kapaa,US,80.58,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,25,Avarua,CK,84.25,-21.2078,-159.7750,Paradise Inn


In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Heat map using the preferred hotel data
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(hotel_df[["Lat","Lng"]],
                    weights=hotel_df["Max Temp"],
                    dissipating=False,
                    max_intensity=300,
                    point_radius=3)

marker_layer = gmaps.marker_layer(hotel_df[["Lat","Lng"]],info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))